# SEARCH ENGINE WITH TOOLS AND AGENTS 

In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [4]:
tools = [wiki,arxiv]

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [8]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()
mongo_uri = os.getenv("MONGODB_URI")
client = MongoClient(mongo_uri)
db = client.practicedb
collection = db.collection1
ATLAS_VECTOR_SEARCH_INDEX_NAME = "collection1-vector-search-index"
vectorstore = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine"
)
vectorstore.create_vector_search_index(dimensions=384)

In [10]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorstore.add_documents(documents=documents)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['MongoDBAtlasVectorSearch', 'HuggingFaceEmbeddings'], vectorstore=<langchain_mongodb.vectorstores.MongoDBAtlasVectorSearch object at 0x00000286AA31DB70>, search_kwargs={})

--> Now we must convert this retriever to a tool

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about langsmith")

retriever_tool.name

'langsmith-search'

In [12]:
tools.append(retriever_tool)
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\HP\\anaconda3\\envs\\my_env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000286AF8636D0>, retriever=VectorStoreRetriever(tags=['MongoDBAtlasVectorSearch', 'HuggingFaceEmbeddings'], vectorstore=<langchain_mongodb.vectorstores.MongoDBAtlasVectorSe

In [19]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=groq_api_key, model_name="Llama3-8b-8192")


In [20]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\HP\anaconda3\envs\my_env\lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)#verbose=True to interaction
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [23]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with Lan

{'input': 'Tell me about Langsmith',
 'output': "I think I can provide a direct response now. LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features for observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, allowing them to ship quickly and with confidence. LangSmith is framework-agnostic and can be used with or without LangChain's open-source frameworks, langchain and langgraph."}

In [24]:
agent_executor.invoke({"input":"Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Statistical algorithms'}`


Page: Computational statistics
Summary: Computational statistics, or statistical computing, is the study which is the intersection of statistics and computer science, and refers to the statistical methods that are enabled by using computational methoHere is a direct response:

Machine learning is a field of study that is closely related to computational statistics. It involves the development and application of statistical algorithms that can learn from data and make predictions or decisions. These algorithms are often implemented using computational methods and are typical

{'input': 'Tell me about machine learning',
 'output': 'Here is a direct response:\n\nMachine learning is a field of study that is closely related to computational statistics. It involves the development and application of statistical algorithms that can learn from data and make predictions or decisions. These algorithms are often implemented using computational methods and are typically used to analyze large datasets and make predictions about new, unseen data.'}

In [28]:
agent_executor.invoke({"input":"What's the research paper attention is all you need about?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `wikipedia` with `{'query': 'transformer deep learning'}`


Page: Transformer (deep learning architecture)
Summary: The transformer is a deep learning architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted inHere's the next step:

The transformer is a deep learning architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector. This vector is then fed into a self-attention mechanism, which allows the model to weigh

{'input': "What's the research paper attention is all you need about?",
 'output': "Here's the next step:\n\nThe transformer is a deep learning architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector. This vector is then fed into a self-attention mechanism, which allows the model to weigh the importance of each token relative to others.\n\nSo, it seems that the transformer is a neural network architecture that's particularly well-suited for natural language processing tasks, such as machine translation, text summarization, and question answering. It's because of its ability to model complex contextual relationships between words in a sentence that the transformer has become so widely used in many areas of NLP."}